In [ ]:
from keras.applications import VGG16
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
# Preprocess the data
train_images = train_images / 255.0
test_images = test_images / 255.0
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [ ]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add new fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

In [ ]:
# Create the final model for training
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Generate predictions on the test set
test_predictions = model.predict(test_images)
predicted_labels = np.argmax(test_predictions, axis=1)

In [ ]:
# Plot a random sample of test images with their predicted labels
plt.figure(figsize=(10, 10))
for i in range(25):
    idx = np.random.randint(0, len(test_images))
    image = test_images[idx]
    true_label = np.argmax(test_labels[idx])
    predicted_label = predicted_labels[idx]
    plt.subplot(5, 5, i+1)
    plt.imshow(image)
    plt.axis('off')
    plt.title(f'True: {true_label}\nPredicted: {predicted_label}')
plt.tight_layout()
plt.show()